In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Models:
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

# Python utilities:
import time
import os


### 探索数据

In [ ]:
df = pd.read_csv('../input/GiveMeSomeCredit/cs-training.csv')
tf = pd.read_csv('../input/GiveMeSomeCredit/cs-test.csv')
list(df)

我们还想知道数据集的平衡程度，所以我们检查哪些部分是默认的:

In [ ]:
df.SeriousDlqin2yrs.mean()


“如果你超过55岁，需要抵押贷款，重要的是你要知道贷方是否会根据你的年龄拒绝贷款。但年龄可能是你抵押贷款的一个因素。”
所以首先考虑的是年龄问题。
为了解决这个问题，我们需要确认在我们的数据集中，年龄对违约概率有负影响。
然后，我们可以假设我们建立的模型反映了这一趋势，不会仅仅因为某人年纪大了就拒绝贷款。

In [ ]:
age_hist = df['age'].hist(by=df['SeriousDlqin2yrs'], bins=20, layout=(2,1))
age_hist[0].set_xlim((0,100))
age_hist[0].set_title('SeriousDlqin2yrs = 0')
age_hist[1].set_xlim((0,100))
age_hist[1].set_title('SeriousDlqin2yrs = 1')

根据违约的人和没有违约的人的分布可以发现，一般来说，年轻人更容易违约。

接下来看看负债比率。

In [ ]:
df.DebtRatio.describe()


In [ ]:
tf.DebtRatio.describe()

max值过于离谱，检查一下

In [ ]:
df.DebtRatio.quantile([.975])

数据集中2.5%的人负债超过其所拥有资产的3500倍。
需要进一步判断是不是异常值。

In [ ]:
df[df['DebtRatio'] > 3489.025][['SeriousDlqin2yrs','MonthlyIncome']].describe()

这里看到两件特别的事情。
+ 在4000条负债比率为3500的记录中，只有185条记录的月收入值。
+ 有月收入的人似乎月收入要么是1，要么是0。

In [ ]:
df[(df['DebtRatio'] > 3489.025) & (df['SeriousDlqin2yrs'] == df['MonthlyIncome'])].shape[0]

在这185项中，有164项的2年违约率和月收入值相同，说明存在数据输入错误。

第二个问题是，尽管这些人的债务是他们所拥有的几千倍，但他们的违约率并不比一般人高。我们可以得出结论，这些条目一定是数据输入错误，因此我们将从模型中删除它们。

然后关注NumberOfTimes90DaysLate。

In [ ]:
df.groupby('NumberOfTimes90DaysLate').NumberOfTimes90DaysLate.count()

没有人迟17到96倍，但数百人迟到98倍。在这几百份记录:

In [ ]:
df[df['NumberOfTimes90DaysLate'] > 95][['SeriousDlqin2yrs','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTime30-59DaysPastDueNotWorse','NumberOfTimes90DaysLate']].describe()

300人都是30-59天late96/98次，60-89天late96/98次，90+天late96/98次。

这些人的违约率高达55%（整个人口的违约率为6）%。

不能丢掉这些数据。用Winsorize方法将所有96/98替换为20，使它们不是极端异常值，看看是否能改进模型。


最后，考虑对revolving utilization of unsecured lines。它表示的是欠款与信用额度的比率，所以应该不会超过1。首先看接近1的值，看看它们的违约率。我们从0.9到4.0开始:

In [ ]:
df[(df['RevolvingUtilizationOfUnsecuredLines'] > .9) & (df['RevolvingUtilizationOfUnsecuredLines'] <= 4)].SeriousDlqin2yrs.describe()

这两万人的违约率几乎是0.25。

但是RUUL更强的人呢?考虑4到10:

In [ ]:
df[(df['RevolvingUtilizationOfUnsecuredLines'] > 4) & (df['RevolvingUtilizationOfUnsecuredLines'] <= 10)].SeriousDlqin2yrs.describe()

这个地区只有23个记录，但它们的违约率仍然很高。如果考虑RULL>10的

In [ ]:
df[df['RevolvingUtilizationOfUnsecuredLines'] > 10].describe()

这241个人的违约率并不比其他人高，尽管其中一些人的债务是其信用额度的几十万倍。

这些似乎与其他数据不一致，所以我们可以将它们从模型中删除

最后，使用中位数来填充月度收入的缺失值:

In [ ]:
df['MonthlyIncome']=df['MonthlyIncome'].replace(np.nan, df['MonthlyIncome'].median())
tf['MonthlyIncome']=df['MonthlyIncome'].replace(np.nan, df['MonthlyIncome'].median())

缺少值的另一列是Numberofdependencies。50%的非缺失值有0l个依赖项，如果有人将字段留空，很可能是因为没有任何依赖项，这里将用0替换这些值。

### 数据集
基于以上的调查，我构建了一些数据集来改善模型性能:

In [ ]:
# Median Fill, Outliers Removed
removed_debt_outliers = df.drop(df[df['DebtRatio'] > 3489.025].index)
removed_debt_outliers = removed_debt_outliers.fillna(removed_debt_outliers.median())

In [ ]:
#填充测试集
tf=tf.fillna(tf.median())

In [ ]:
# Removed utilization outliers
dfus = removed_debt_outliers.drop(removed_debt_outliers[removed_debt_outliers['RevolvingUtilizationOfUnsecuredLines'] > 10].index)

In [ ]:
# Removed 98s
dfn98 = dfus.copy()
dfn98.loc[dfn98['NumberOfTime30-59DaysPastDueNotWorse'] > 90, 'NumberOfTime30-59DaysPastDueNotWorse'] = 18
dfn98.loc[dfn98['NumberOfTime60-89DaysPastDueNotWorse'] > 90, 'NumberOfTime60-89DaysPastDueNotWorse'] = 18
dfn98.loc[dfn98['NumberOfTimes90DaysLate'] > 90, 'NumberOfTimes90DaysLate'] = 18

### 模型测试
为了能够快速测试模型和数据集，并查看它们彼此之间的比较，封装一个测试类。

In [ ]:
class Tester():
    def __init__(self, target):
        self.target = target
        self.datasets = {}
        self.models = {}
        self.cache = {} 

    def addDataset(self, name, df):
        self.datasets[name] = df.copy()

    def addModel(self, name, model):
        self.models[name] = model
        
    def clearModels(self):
        self.models = {}

    def clearCache(self):
        self.cache = {}
    
    def testModelWithDataset(self, m_name, df_name, sample_len, cv):
        if (m_name, df_name, sample_len, cv) in self.cache:
            return self.cache[(m_name, df_name, sample_len, cv)]

        clf = self.models[m_name]
        
        if not sample_len: 
            sample = self.datasets[df_name]
        else: sample = self.datasets[df_name].sample(sample_len)
            
        X = sample.drop([self.target], axis=1)
        Y = sample[self.target]

        s = cross_validate(clf, X, Y, scoring=['roc_auc'], cv=cv, n_jobs=-1)
        self.cache[(m_name, df_name, sample_len, cv)] = s

        return s

    def runTests(self, sample_len=80000, cv=4):
        # 在所有添加的数据集上测试添加的模型
        scores = {}
        for m_name in self.models:
            for df_name in self.datasets:
                start = time.time()

                score = self.testModelWithDataset(m_name, df_name, sample_len, cv)
                scores[(m_name, df_name)] = score
                
                end = time.time()
                


        print('--- Top 10 Results ---')
        for score in sorted(scores.items(), key=lambda x: -1 * x[1]['test_roc_auc'].mean())[:10]:
            auc = score[1]['test_roc_auc']
            print("%s --> AUC: %0.4f (+/- %0.4f)" % (str(score[0]), auc.mean(), auc.std()))

            
# 在所有模型中使用测试对象
tester = Tester('SeriousDlqin2yrs')

# Y添加数据集举例
tester.addDataset('Drop Missing', df.dropna())

# 添加模型举例
rfc = RandomForestClassifier(n_estimators=15, max_depth = 6, random_state=0)
tester.addModel('Simple Random Forest', rfc)
tester.addModel('Simple SVM', svm.LinearSVC())


tester.runTests()

有了这个类，就回到在数据探索部分中创建的数据集。

In [ ]:
tester.addDataset('Median Fill', df.fillna(df.median()))
tester.addDataset('Median Fill, Outliers Removed', removed_debt_outliers)
tester.addDataset('Removed 98s', dfn98)
tester.addDataset('Removed utilization outliers', dfus)

tester.runTests()

注意到修改后的数据集对SVM的AUC分数有很大的影响，但它们对随机森林的影响没有那么大。然而，在所有模型中，简单地删除缺失的值和修改的数据集之间有一个显著的性能提高。

### 随机森林
随机森林是最好的选择，接下来找出最佳优化参数。

In [ ]:
#已经跑完了不再跑了
# from sklearn.ensemble import RandomForestClassifier

# for i in range(5,10):
#     for j in range(10,20):
#         rfc = RandomForestClassifier(n_estimators=j,max_depth = i, random_state=0)
#         tester.addModel('Random Forest '+'d: '+str(i)+' est: '+str(j)  ,rfc)

# tester.runTests()

所以最好的是
('Random Forest d: 9 est: 15', 'Removed 98s') --> AUC: 0.8660 (+/- 0.0077)

In [ ]:
y=dfn98["SeriousDlqin2yrs"]

In [ ]:
features=['RevolvingUtilizationOfUnsecuredLines',
 'age',
 'NumberOfTime30-59DaysPastDueNotWorse',
 'DebtRatio',
 'MonthlyIncome',
 'NumberOfOpenCreditLinesAndLoans',
 'NumberOfTimes90DaysLate',
 'NumberRealEstateLoansOrLines',
 'NumberOfTime60-89DaysPastDueNotWorse',
 'NumberOfDependents']

In [ ]:
X=pd.get_dummies(dfn98[features])

In [ ]:
X_test=pd.get_dummies(tf[features])

In [ ]:
model=RandomForestClassifier(n_estimators=9,max_depth = 15, random_state=0)

In [ ]:
model.fit(X,y)
predictions=model.predict_proba(X_test)[:,1]

In [ ]:
ids = np.arange(1, 101504)

In [ ]:
output=pd.DataFrame({'Id':ids,'Probability':predictions})

In [ ]:
output.to_csv('Give Me Some Credit.csv',index=False)
print("done")